In [10]:
import pickle, os
import requests as reqlib

import pandas as pd
import numpy as np
import cma

from tqdm.notebook import tqdm

In [11]:
# Convert to requests
requests = []

df_reference = pd.DataFrame({
    'request_index': [1, 2, 3, 4],
    'origin_x': [2600152.0184,  2686996.15, 2683189.54, 2759398.33],
    'origin_y': [1199620.154,  1250908.71, 1248065.92, 1191228.778],
    'destination_x': [2683802.83,  2683802.83, 2789162.6993, 2789162.6993],
    'destination_y': [1248309.94,  1248309.94, 1151920.922, 1151920.922],
    'transfers': [1,2,4, 4],
    'departure_time': [7*3600, 10*3600, 6*3600, 7*3600] 
})

for index, row in df_reference.iterrows():
    requests.append({
        "request_index": int(row["request_index"]),
        "origin_x": row["origin_x"],
        "origin_y": row["origin_y"],
        "destination_x": row["destination_x"],
        "destination_y": row["destination_y"],
        "departure_time_s": row["departure_time"]
    })

In [12]:
requests

[{'request_index': 1,
  'origin_x': np.float64(2600152.0184),
  'origin_y': np.float64(1199620.154),
  'destination_x': np.float64(2683802.83),
  'destination_y': np.float64(1248309.94),
  'departure_time_s': np.float64(25200.0)},
 {'request_index': 2,
  'origin_x': np.float64(2686996.15),
  'origin_y': np.float64(1250908.71),
  'destination_x': np.float64(2683802.83),
  'destination_y': np.float64(1248309.94),
  'departure_time_s': np.float64(36000.0)},
 {'request_index': 3,
  'origin_x': np.float64(2683189.54),
  'origin_y': np.float64(1248065.92),
  'destination_x': np.float64(2789162.6993),
  'destination_y': np.float64(1151920.922),
  'departure_time_s': np.float64(21600.0)},
 {'request_index': 4,
  'origin_x': np.float64(2759398.33),
  'origin_y': np.float64(1191228.778),
  'destination_x': np.float64(2789162.6993),
  'destination_y': np.float64(1151920.922),
  'departure_time_s': np.float64(25200.0)}]

In [13]:
routing_endpoint = "http://localhost:8054/router/transit"

modes = [c.replace("legs_", "") for c in df_reference.columns if c.startswith("legs_")]
maximum_transfers = df_reference["transfers"].max()

variables = [
    { "name": "rail_u_h", "initial": -1.0 },
    { "name": "subway_u_h", "initial": -1.0, "fixed": True },
    { "name": "bus_u_h", "initial": -5.0 },
    { "name": "tram_u_h", "initial": -1.0 },
    { "name": "other_u_h", "copy": "bus_u_h" },
    { "name": "wait_u_h", "initial": -10.0 },
    { "name": "walk_u_h", "initial": -1.0 },
    { "name": "transfer_u", "initial": -2.0 }
]

# Prepare querying the routing server
def query_endpoint(requests, utilities):
    response = reqlib.post(routing_endpoint, json = {
        "batch": requests,
        "utilities": utilities
    })

    assert response.status_code == 200

    print(response)

    df_response = { 
        "request_index": [],
        "transfers": []
    }

    for mode in modes:
        df_response["legs_{}".format(mode)] = []

    for row in response.json():
        df_response["request_index"].append(row["request_index"])
        df_response["transfers"].append(np.minimum(row["transfers"], maximum_transfers))
        
        for mode in modes:
            if mode in row["vehicle_legs_by_mode"]:
                df_response["legs_{}".format(mode)].append(row["vehicle_legs_by_mode"][mode])
            else:
                df_response["legs_{}".format(mode)].append(0)
    
    return pd.DataFrame(df_response)

In [14]:
# Extend with index information for CMA-ES evaluation
variables_map = { v["name"]: v for v in variables }

active_index = 0

# First find active variables
for variable in variables:
    if "fixed" in variable or "copy" in variable: 
        continue

    variables_map[variable["name"]] = variable
    
    variable["index"] = active_index
    variable["optimized"] = True
    active_index += 1

# Then treat fixed variables and those copying from others
for variable in variables:
    if "fixed" in variable:
        variable["index"] = None
    
    if "copy" in variable:
        assert not "initial" in variable
        variable["initial"] = variables_map[variable["copy"]]["initial"]
        variable["index"] = variables_map[variable["copy"]]["index"]

In [15]:
initial = []
bounds = [[], []]

for variable in variables:
    if "optimized" in variable:
        initial.append(variable["initial"])
        bounds[0].append(-np.inf)
        bounds[1].append(0.0)

def prepare_utilities(values):
    utilities = {}
    
    for variable in variables:
        if variable["index"] is not None:
            utilities[variable["name"]] = values[variable["index"]]
        else:
            utilities[variable["name"]] = variable["initial"]
    
    return utilities

candidates = [initial]

for candidate in candidates:
    utilities = prepare_utilities(candidate)

In [16]:
maximum_batch_size = 400
df_response = []
batch_index = 0

with tqdm(total=len(requests)) as progress:
    while batch_index * maximum_batch_size < len(requests):
        batch = requests[batch_index * maximum_batch_size : (batch_index + 1) * maximum_batch_size]
        
        # Convert numpy types to native Python
        batch_serializable = [
            {
                k: v.item() if isinstance(v, (np.integer, np.floating)) else v
                for k, v in request.items()
            }
            for request in batch
        ]
        
        response = reqlib.post(routing_endpoint, json={
            "batch": batch_serializable,
            "utilities": utilities
        })

        print(response.json())
        
        df_response.append(pd.DataFrame.from_records(response.json()))
        progress.update(len(batch))
        batch_index += 1

df_response = pd.concat(df_response)

  0%|          | 0/4 [00:00<?, ?it/s]

[{'request_index': 1, 'in_vehicle_travel_time_min': 60.0, 'in_vehicle_travel_time_by_mode_min': {'rail': 60.0}, 'in_vehicle_distance_km': 118.74070234165853, 'in_vehicle_distance_by_mode_km': {'rail': 118.74070234165853}, 'vehicle_legs_by_mode': {'rail': 1}, 'modes_sequence': ['rail'], 'access_walk_time_min': 4.066666666666666, 'egress_walk_time_min': 17.316666666666666, 'transfer_walk_time_min': 0.0, 'access_walk_distance_km': 0.2244981558107801, 'egress_walk_distance_km': 0.9588562414355055, 'transfer_walk_distance_km': 0.0, 'initial_wait_time_min': 5.933333333333334, 'transfer_wait_time_min': 0.0, 'transfers': 0, 'total_travel_time_min': 87.31666666666666, 'arrivalTime_s': 30439.0, 'is_only_walk': False}, {'request_index': 2, 'in_vehicle_travel_time_min': 18.0, 'in_vehicle_travel_time_by_mode_min': {'tram': 18.0}, 'in_vehicle_distance_km': 6.6963645452913, 'in_vehicle_distance_by_mode_km': {'tram': 6.6963645452913}, 'vehicle_legs_by_mode': {'tram': 1}, 'modes_sequence': ['tram'], 'a

In [17]:
df_response

,request_index,in_vehicle_travel_time_min,in_vehicle_travel_time_by_mode_min,in_vehicle_distance_km,in_vehicle_distance_by_mode_km,vehicle_legs_by_mode,modes_sequence,access_walk_time_min,egress_walk_time_min,transfer_walk_time_min,access_walk_distance_km,egress_walk_distance_km,transfer_walk_distance_km,initial_wait_time_min,transfer_wait_time_min,transfers,total_travel_time_min,arrivalTime_s,is_only_walk
0,1,60.000000,{'rail': 60.0},118.740702,{'rail': 118.74070234165853},{'rail': 1},[rail],4.066667,17.316667,0.000000,0.224498,0.958856,0.000,5.933333,0.000000,0,87.316667,30439.0,False
1,2,18.000000,{'tram': 18.0},6.696365,{'tram': 6.6963645452913},{'tram': 1},[tram],6.216667,0.016667,0.000000,0.343977,0.000012,0.000,1.783333,0.000000,0,26.016667,37561.0,False
2,3,225.966667,{'rail': 225.96666666666667},192.682789,{'rail': 192.68278877316948},{'rail': 3},"[rail, rail, rail]",2.383333,12.616667,3.983333,0.131136,0.698601,0.000,3.616667,24.033333,2,272.600000,37956.0,False
3,4,134.983333,"{'bus': 11.983333333333333, 'rail': 123.0}",73.248591,"{'bus': 6.331915847264327, 'rail': 66.91667559...","{'bus': 1, 'rail': 2}","[rail, rail, bus]",23.466667,11.000000,5.983333,1.298980,0.608950,0.326,10.533333,18.016667,2,203.983333,37439.0,False


In [18]:
utilities

{'rail_u_h': -1.0,
 'subway_u_h': -1.0,
 'bus_u_h': -5.0,
 'tram_u_h': -1.0,
 'other_u_h': -5.0,
 'wait_u_h': -10.0,
 'walk_u_h': -1.0,
 'transfer_u': -2.0}